# Alpha Range Check
THe range for the 8 fitting parameters seem to fluctuate between rounds. Some are exceptionally different than the rest. This notebook should print out the min/max ranges for all the alphas in all the rounds.

## Notes
## Alpha0
This is the bias term and will respond to any multipliers in the data. The other alphas however, do not. Which means, if there was an unaccounted system wide gain, alpha0 is the only one which would be affected.

## Settings
It might be helpful if there was a way to pull the settings(txt) file/additional information associated with each round. (Maybe later)

### Extreme Outliers
For 4 rounds, the alpha values are way way off. All of them are from sp2021 and 3/4 of them are 4 detector setups.

In [1]:
from inverse_modelling_tfo.data.intensity_interpolation import interpolate_exp_chunk, get_interpolate_fit_params
from inverse_modelling_tfo.data.interpolation_function_zoo import unity_at_zero_interpolation
from typing import List, Union
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from TFO_dataset import SheepData

In [2]:
def prepare_patient_ppg(ppg_data : pd.DataFrame, sample_number : Union[int, List], SDD = [15, 30, 45, 70, 100], weights=[1.0, -1.0]) -> np.ndarray:
    """Create fitting parameters from PPG data. 

    Args:
        ppg_data (pd.DataFrame): PPG data DataFrame. You can feed data directly from the the TFO_dataset package.
        (Note: This should ideally be the optically normalized data)
        sample_number (int): which sample to choose. You can either pass a single integer or an array
        SDD (_type_, optional): Detector distances in TFO device(in mm). Defaults to SDD=[15, 30, 45, 70, 100].
        weights: Exponential weights applied during fitting to the detectors
    """
    # The code is generalized to run on any array. make necessary conversions 
    if isinstance(sample_number, int):
        sample_number = [sample_number]
    
    patient_features = []
    for sample_point in sample_number:
        # Pick a point in time
        spatial_intensity = ppg_data.iloc[sample_point]
        # Reshape ppg data to fit the format
        spatial_intensity_wv1 = pd.DataFrame(data={
            'SDD' : SDD,
            'Intensity' : spatial_intensity.to_numpy()[:len(SDD)]
        })
        spatial_intensity_wv2 = pd.DataFrame(data={
            'SDD' : SDD,
            'Intensity' : spatial_intensity.to_numpy()[5: 5 + len(SDD)]
        })
        # alpha_wv1 = interpolate_exp_chunk(spatial_intensity_wv1, weights=weights, return_alpha=True).flatten()
        # alpha_wv2 = interpolate_exp_chunk(spatial_intensity_wv2, weights=weights, return_alpha=True).flatten()
        alpha_wv1 = unity_at_zero_interpolation(spatial_intensity_wv1, weights=weights).flatten()
        alpha_wv2 = unity_at_zero_interpolation(spatial_intensity_wv2, weights=weights).flatten()
        # patient_features.append([alpha_wv1[0], alpha_wv2[0], alpha_wv1[1], alpha_wv2[1], alpha_wv1[2], alpha_wv2[2], alpha_wv1[3], alpha_wv2[3]])
        patient_features.append([*alpha_wv1, *alpha_wv2])
        
    return np.array(patient_features)

There is some weird issue when the TFO probe has only 4 detectors. Although the fitting does account for this, (By only choosing the first 4 channels and ignoring ch5 for both wavelenghts). The fitting is still an outlier

In [10]:
sample_spacing = 1000     # Choosing all points would be very slow
shd = SheepData('iq_demod_optical')
all_tags = shd.get_available_tags()
weights = [0, 1]
print(len(all_tags))

comparison_table = pd.DataFrame(columns=['Sheep', 'Round', 'Year', 'Info', 'a0_min', 'a0_max', 'a1_min', 'a1_max', 'a2_min', 'a2_max', 'a3_min', 'a3_max'])
for tag_index, tag in enumerate(all_tags):
    # Not all rounds have avaialble normalizations constants/ignore those
    try:
        data = shd.get_data_from_tag(tag)
        sdd = shd.get_sdd_distance(tag)
    except:
        continue
    new_row = {'Sheep' : tag['experiment_number'], 'Round' : tag['experiment_round'], 'Year': tag['experiment_year_prefix'], 
               'Info': tag['additional_info']}
    features = prepare_patient_ppg(data, np.arange(0, len(data), sample_spacing), sdd, weights)
    for feature_index in range(0, features.shape[1]//2):  # Only wv1 features
        feature_max = np.nanmax(features[:, feature_index])
        feature_min = np.nanmin(features[:, feature_index])
        new_row[f'a{feature_index}_min'] = feature_min
        new_row[f'a{feature_index}_max'] = feature_max
    new_row = pd.DataFrame(new_row, index=[tag_index])
    comparison_table = pd.concat([comparison_table, new_row])

pd.set_option('display.max_rows', None)
comparison_table
# PS: NaN could actually also mean that fitting param. just does not exist.

86


/home/rraiyan/personal_projects/tfo_inverse_modelling/inverse_modelling_tfo/data/interpolation_function_zoo.py:32: RuntimeWarning: invalid value encountered in log10
  Y = np.log10(input_y).reshape(-1, 1)
/home/rraiyan/personal_projects/tfo_inverse_modelling/inverse_modelling_tfo/data/interpolation_function_zoo.py:32: RuntimeWarning: invalid value encountered in log10
  Y = np.log10(input_y).reshape(-1, 1)
/home/rraiyan/personal_projects/tfo_inverse_modelling/inverse_modelling_tfo/data/interpolation_function_zoo.py:32: RuntimeWarning: invalid value encountered in log10
  Y = np.log10(input_y).reshape(-1, 1)


This round does not have all feedback resistors defined. Ch5 Resistance assumed 1.0
This round does not have all feedback resistors defined. Ch5 Resistance assumed 1.0
This round does not have all feedback resistors defined. Ch5 Resistance assumed 1.0
This round does not have all feedback resistors defined. Ch5 Resistance assumed 1.0
This round does not have all feedback resistors defined. Ch5 Resistance assumed 1.0
This round does not have all feedback resistors defined. Ch5 Resistance assumed 1.0
This round does not have all feedback resistors defined. Ch5 Resistance assumed 1.0
This round does not have all feedback resistors defined. Ch5 Resistance assumed 1.0


,Sheep,Round,Year,Info,a0_min,a0_max,a1_min,a1_max,a2_min,a2_max,a3_min,a3_max
0,3,1,su2020,,6.062233,8.643252,-31.065740,-24.894219,7.987567,12.166279,NaN,NaN
1,3,1,su2020,Baseline,7.592309,9.040473,-32.000697,-28.352175,10.263007,12.709829,NaN,NaN
2,3,2,su2020,,-41.995702,-7.698511,9.467593,93.559143,-66.148386,-13.517724,NaN,NaN
3,3,2,su2020,Baseline,-18.863110,-14.788492,29.049215,37.773413,-31.109607,-26.000479,NaN,NaN
4,4,1,su2020,,-18.907378,-9.815638,15.729874,37.834756,-31.186326,-17.081206,NaN,NaN
5,4,1,su2020,Baseline,-10.519679,-8.781172,13.515837,17.720337,-18.322436,-15.683721,NaN,NaN
6,4,2,su2020,,-13.350637,-8.385149,8.975347,20.290988,-18.700505,-11.819138,NaN,NaN
7,4,2,su2020,Baseline,-11.882922,-9.420482,11.063786,17.083254,-16.749783,-12.966414,NaN,NaN
8,4,3,su2020,,-11.088966,-8.817495,10.847636,16.084022,-15.994032,-12.750582,NaN,NaN
9,4,3,su2020,Baseline,-10.287381,-8.285025,10.094347,13.910311,-14.385470,-12.091372,NaN,NaN


In [14]:
fitting_param_columns = comparison_table[list(filter(lambda X: 'a' in X, comparison_table.columns))]
fitting_param_columns.describe()

,a0_min,a0_max,a1_min,a1_max,a2_min,a2_max
count,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000
mean,-4.882116,-0.116837,-5.924911,6.267664,-11.770486,-3.763459
std,10.325571,7.999756,18.755971,24.170940,15.082098,11.704078
min,-45.266375,-25.797289,-63.638610,-52.248107,-71.187985,-38.024150
25%,-9.479135,-5.966819,-16.852323,-5.906865,-16.552948,-11.824409
50%,-4.748019,0.899754,-7.505965,5.434611,-10.627891,-3.659185
75%,0.604948,5.525183,8.032832,15.430938,-3.892194,2.721257
max,15.428644,20.172955,50.466466,101.792209,27.499482,34.741527


In [15]:
# Print 4 detector rounds - For cross check!
for tag in all_tags:
    if tag['additional_info'] != '':
        continue
    # Not all rounds have avaialble normalizations constants/ignore those
    try:
        sdd = shd.get_sdd_distance(tag)
        if len(sdd) == 4:
            print(tag)
    except:
        continue

{'experiment_number': 3, 'experiment_round': 1, 'experiment_year_prefix': 'sp2021', 'additional_info': '', 'data_version': 'iq_demod_optical', 'experiment_type': 'sheep'}
{'experiment_number': 3, 'experiment_round': 2, 'experiment_year_prefix': 'sp2021', 'additional_info': '', 'data_version': 'iq_demod_optical', 'experiment_type': 'sheep'}
{'experiment_number': 4, 'experiment_round': 1, 'experiment_year_prefix': 'sp2021', 'additional_info': '', 'data_version': 'iq_demod_optical', 'experiment_type': 'sheep'}
{'experiment_number': 4, 'experiment_round': 2, 'experiment_year_prefix': 'sp2021', 'additional_info': '', 'data_version': 'iq_demod_optical', 'experiment_type': 'sheep'}
{'experiment_number': 4, 'experiment_round': 3, 'experiment_year_prefix': 'sp2021', 'additional_info': '', 'data_version': 'iq_demod_optical', 'experiment_type': 'sheep'}
{'experiment_number': 5, 'experiment_round': 1, 'experiment_year_prefix': 'sp2021', 'additional_info': '', 'data_version': 'iq_demod_optical', 'e

## Rounds with consistent alphas between Baseline and Regular Round
### Summer 2020
3 - 1  
3 - 2  
4 - 1  
4 - 2  
4 - 3  
5 - 1 (Inconsistent)  
5 - 2  

### Spring 2021
1 - 1  
1 - 2  
1 - 3  
2 - 1  
2 - 2  
2 - 3  
  
### Spring 2022
1 - 1  
1 - 2  
1 - 3  
2 - 1  
2 - 2  
2 - 3  
4 - 1  
4 - 2  
7 - 1  
7 - 3  
10 - 1  
10 - 2  
11 - 1  
11 - 2  